In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
MAX_TIME_STEP = 30

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
df_raw = pd.read_csv("/content/drive/MyDrive/data/archive(1)/LeagueofLegends.csv")
df_raw = df_raw[df_raw['gamelength'] >= MAX_TIME_STEP]
df_raw.reset_index(drop = True, inplace = True)
print(len(df_raw))

6384


In [21]:
df = pd.read_csv("/content/drive/MyDrive/data/archive(1)/LeagueofLegends.csv")
df = df[df['gamelength'] >= MAX_TIME_STEP]
df.reset_index(drop = True, inplace = True)
matches = len(df)
print(matches)

6384


In [22]:
def count_item(items):
    count = np.zeros(MAX_TIME_STEP, dtype=np.int8)
    for timestep in range(MAX_TIME_STEP) :
        for item in items:
            if item[0] <= timestep + 1:
                count[timestep] += 1
    return count

In [23]:
from ast import literal_eval
df['golddiff'] = df['golddiff'].apply(literal_eval)
blue = ['bDragons', 'bBarons', 'bHeralds', 'bTowers', 'bInhibs', 'bKills']
red = ['rDragons', 'rBarons', 'rHeralds', 'rTowers', 'rInhibs', 'rKills']
diffs = ['dragondiff', 'barondiff', 'heralddiff', 'towerdiff', 'inhibitordiff', 'killdiff']
for r, b, d in zip(blue, red, diffs):
    df[b] = df[b].apply(literal_eval)
    df[r] = df[r].apply(literal_eval)

    df[b] = df[b].apply(count_item)
    df[r] = df[r].apply(count_item)
    df[d] = df[b] - df[r]

In [24]:
df_raw["bKills"] = df_raw["bKills"].apply(literal_eval)
df_raw["rKills"] = df_raw["rKills"].apply(literal_eval)

In [25]:
bkls = df_raw["bKills"]
btags = df_raw["blueTeamTag"]
bjng = df_raw["blueJungle"]
blue_ganks = []
blue_count = []
for jnglr, res, tag in zip(bjng, bkls, btags):
    #print([(tag +" " + jnglr) in res[i][3] and len(res[i][3]) <= 3 for i in range(len(res))])
    tag = str(tag)
    jnglr = str(jnglr)
    num_ganks = [(tag + " " + jnglr) in res[i][3] and len(res[i][3]) < 2 for i in range(len(res))].count(True)
    blue_ganks.append(num_ganks)
    blue_count.append(len(res))

In [26]:
rkls = df_raw["rKills"]
rtags = df_raw["redTeamTag"]
rjng = df_raw["redJungle"]
red_ganks = []
red_count = []
for jnglr, res, tag in zip(rjng, rkls, rtags):
    #print([(tag +" " + jnglr) in res[i][3] and len(res[i][3]) <= 3 for i in range(len(res))])
    tag = str(tag)
    jnglr = str(jnglr)
    num_ganks = [(tag + " " + jnglr) in res[i][3] and len(res[i][3]) < 2 for i in range(len(res))].count(True)
    red_ganks.append(num_ganks)
    red_count.append(len(res))

In [27]:
df_raw['gamelength']

0       40
1       38
2       40
3       41
4       35
        ..
6379    36
6380    34
6381    39
6382    35
6383    42
Name: gamelength, Length: 6384, dtype: int64

In [28]:
ganks_wtime = []
for jnglr, res, tag in zip(rjng, rkls, rtags):
  tag = str(tag)
  jnglr = str(jnglr)
  arr = np.array(res)
  bool_arr = np.array([(tag + " " + jnglr) in res[i][3] and len(res[i][3]) < 3 for i in range(len(res))])
  if len(bool_arr) == 0:
    continue
  if len(arr[bool_arr]) == 0:
    continue
  ganks_wtime.append(arr[bool_arr])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """


In [29]:
times = []
for gank in ganks_wtime:
  temp = []
  for k in range(gank.shape[0]):
    temp.append(gank[k][0])
  times.append(temp)

In [30]:
times

[[18.784],
 [36.418],
 [21.115, 26.753],
 [5.866, 23.927, 30.799, 30.856],
 [30.763],
 [7.606],
 [13.562, 16.25, 16.747, 18.887, 25.737],
 [38.318],
 [6.611],
 [6.894, 6.973, 16.966],
 [36.786, 39.007],
 [32.492, 34.518],
 [6.876, 18.437, 35.673],
 [10.083, 15.689, 16.281, 30.001, 33.592, 33.747, 33.896, 34.067, 37.24],
 [18.491],
 [3.475, 11.403, 31.043, 35.016],
 [9.344, 11.418],
 [16.67, 30.166],
 [11.488, 40.524],
 [13.658, 20.159],
 [6.816],
 [7.441, 11.057, 13.101, 23.121, 26.042],
 [6.453, 15.804, 33.745, 34.32],
 [8.643],
 [5.304, 6.342, 11.494, 15.005, 28.496],
 [13.329, 28.662, 28.756, 39.317, 39.344],
 [25.021, 26.845],
 [8.822, 13.028],
 [19.164, 20.685, 33.821],
 [28.354, 37.157],
 [29.996],
 [32.235],
 [5.967, 26.77],
 [14.594, 37.094, 51.439],
 [10.414, 10.505, 10.679, 21.612, 29.903],
 [10.941, 11.002, 26.499],
 [14.554, 26.841, 26.952],
 [28.843],
 [10.236, 13.019],
 [11.723],
 [17.548],
 [9.126, 28.396, 32.981],
 [16.052, 29.744],
 [33.854],
 [27.056],
 [26.195, 32.38

In [10]:
df["bGanks"] = blue_ganks
df["rGanks"] = red_ganks
df["bkCount"]= blue_count
df["rkCount"]= red_count

In [11]:
stats = ['golddiff','dragondiff', 'barondiff', 'heralddiff', 'towerdiff', 'inhibitordiff', 'killdiff']
x = df[stats]
y = df['bResult']

In [ ]:
from sklearn.preprocessing import StandardScaler

data = {}
scalers = {}
for stat in stats:
    scalers[stat] = StandardScaler()
    for row in df[stat]:
        scalers[stat].partial_fit(np.asanyarray(row).reshape(-1, 1))
    data[stat] = [scalers[stat].transform(np.asanyarray(row).reshape(-1, 1)).reshape(-1) for row in df[stat]]

num_features = len(data)
print(f'# of features per timestep: {num_features}')

# of features per timestep: 7


In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
model = Sequential([
    Dense(32, activation='relu', input_shape=(7,)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [ ]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])